
# Text Classification using LSTM

This Code Template is for Text Classification using Long short-term memory in python

### Required Packages

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import nltk
import re
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
pip install tensorflow==2.1.0

### Initialization

Filepath of CSV file

In [3]:
df=pd.read_csv('')

### Data Fetching

Pandas is an open-source, BSD-licensed library providing high-performance, easy-to-use data manipulation and data analysis tools.

We will use panda's library to read the CSV file using its storage path.And we use the head function to display the initial row or entry.

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
###Drop Nan Values
df=df.dropna()


In [6]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [7]:
## Get the Dependent features
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
tf.__version__

'2.1.0'

In [11]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [12]:
messages=X.copy()

In [13]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:
messages.reset_index(inplace=True)

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Data Preprocessing

Since the majority of the machine learning models in the Sklearn library doesn't handle string category data and Null value, we have to explicitly remove or replace null values. The below snippet have functions, which removes the null value if any exists. And convert the string classes data in the datasets by encoding them to integer classes.

In [16]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [30]:
corpus[1:10]

['flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

### Embedding Representation

In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2726  995  208]
 [   0    0    0 ... 3243 4231 2733]
 [   0    0    0 ... 3098 4278 1619]
 ...
 [   0    0    0 ... 3138 3806 4753]
 [   0    0    0 ... 4081 2593 2354]
 [   0    0    0 ... 2082 4030 1513]]


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4791,
        486, 1407, 1043, 3836, 4851, 3975, 2726,  995,  208], dtype=int32)

### Model
#LSTM
The LSTM model will learn a function that maps a sequence of past observations as input to an output observation. As such, the sequence of observations must be transformed into multiple examples from which the LSTM can learn.

Refer [API](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21) for the parameters

In [21]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
len(embedded_docs),y.shape

(18285, (18285,))

In [23]:

X_final=np.array(embedded_docs)
y_final=np.array(y)

In [24]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [25]:

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [26]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 12s 966us/sample - loss: 0.3515 - accuracy: 0.8399 - val_loss: 0.2079 - val_accuracy: 0.9100
Epoch 2/10
12250/12250 [==============================] - 9s 737us/sample - loss: 0.1469 - accuracy: 0.9419 - val_loss: 0.1934 - val_accuracy: 0.9190
Epoch 3/10
12250/12250 [==============================] - 9s 757us/sample - loss: 0.1000 - accuracy: 0.9633 - val_loss: 0.2075 - val_accuracy: 0.9223
Epoch 4/10
12250/12250 [==============================] - 9s 747us/sample - loss: 0.0636 - accuracy: 0.9776 - val_loss: 0.2712 - val_accuracy: 0.9198
Epoch 5/10
12250/12250 [==============================] - 9s 732us/sample - loss: 0.0381 - accuracy: 0.9879 - val_loss: 0.3000 - val_accuracy: 0.9165
Epoch 6/10
12250/12250 [==============================] - 9s 739us/sample - loss: 0.0200 - accuracy: 0.9937 - val_loss: 0.3774 - val_accuracy: 0.9160
Epoch 7/10
12250/12250 [==========================

### Performance Metrics And Accuracy

In [27]:
y_pred=model.predict_classes(X_test)

In [28]:
confusion_matrix(y_test,y_pred)

array([[3111,  308],
       [ 244, 2372]])

## Model Accuracy

In [29]:

accuracy_score(y_test,y_pred)

0.9085335542667772

#### Creator: Ageer Harikrishna , Github: [Profile](https://github.com/ageerHarikrishna)